### 은닉 마르코프 모형 (HMM) - 기초

In [1]:
!pip install numpy==1.21

강의 슬라이드의 HMM 모형을 만들어 본다. 

![](https://drive.google.com/uc?export=view&id=1yJcAnq6u2-lyGXsMvgEJxH9J8Ek0IrPf)

In [2]:
# 다음 라이브러리를 설치한다.
!pip install pomegranate

In [3]:
# 라이브러리를 불러온다.
import numpy as np
from pomegranate import State, HiddenMarkovModel, DiscreteDistribution

#### 1. 모델 파라미터 설정:

1.1. 상태의 초기 확률 (Initial Probabilities):

|        | Prob |
|: --- :|: --- :|
| $Sunny$  | 0.50 |
|$Rainny$  | 0.50 |

1.2. 전이 확률 (Transition Probabilities):

|   | $Sunny$ | $Rainny$ |
|:---:|:---:|:---:|
|$Sunny$| 0.80 | 0.40 |
|$Rainny$| 0.20 | 0.60 | 

1.3. 출력 확률 (Emission Probabilities):

|    |  $Sunny$  | $Rainny$ |
|: ---: |: --- :|: ---: |
| $Game$ |   0.10  | 0.80 |
| $Fitness$ | 0.90 | 0.20 |


#### 2. 모델 생성:

In [4]:
# 모델 생성.
my_hmm = HiddenMarkovModel(name="my_model")

In [5]:
# 상태와 출력확률 P(O | S) 설정.
Sunny_emissions = DiscreteDistribution({"Game": 0.1, "Fitness": 0.9}) #출력 확률 정의 dict
Sunny_state = State(Sunny_emissions, name="Sunny_state")

Rainny_emissions = DiscreteDistribution({"Game": 0.8, "Fitness": 0.2})
Rainny_state = State(Rainny_emissions, name="Rainny_state")

# 모델에 상태 추가.
my_hmm.add_states(Sunny_state)
my_hmm.add_states(Rainny_state)
#my_hmm.add_states([Sunny_state,Rainny_state])

In [6]:
# 초기 확률 설정.
my_hmm.add_transition(my_hmm.start, Sunny_state, 0.5)
my_hmm.add_transition(my_hmm.start, Rainny_state, 0.5)

In [7]:
# 전이 확률 P(S'|S) 설정.
my_hmm.add_transition(Sunny_state, Sunny_state, 0.8)  # 80% Sunny -> Sunny
my_hmm.add_transition(Sunny_state, Rainny_state, 0.2)  # 20% Sunny -> Rainny

my_hmm.add_transition(Rainny_state, Sunny_state, 0.4)  # 40% Rainny -> Sunny
my_hmm.add_transition(Rainny_state, Rainny_state, 0.6)  # 60% Rainny -> Sunny

In [8]:
# 모델 최종 완성.
my_hmm.bake()

In [9]:
# 모델의 구조 출력.
print("Edge counts: ",my_hmm.edge_count()) 
print("Node counts: ",my_hmm.node_count()) 

Edge counts:  6
Node counts:  4


In [10]:
# 입력된 상태의 이름 확인.
for s in my_hmm.states:
    print(s.name)

Rainny_state
Sunny_state
my_model-start
my_model-end


#### 3. 모델을 사용한 예측 (Inference):

In [11]:
# 다음과 같은 시퀀스가 관찰 되었다 전제한다.
observations = ["Game", "Fitness", "Game"] #O의 값

In [12]:
# Viterbi 알고리즘으로 가장 우도 (likelihood)가 높은 경로를 계산한다.
viterbi_likelihood, viterbi_path = my_hmm.viterbi(observations)

In [13]:
# The likely weather.
weather_pred = []
for s in viterbi_path[1:]:
    weather_pred.append(s[1].name)
print("제일 유력한 날씨 : {}".format(weather_pred))
print("우도 함수의 값   : {:0.4f}".format(np.exp(viterbi_likelihood)))

제일 유력한 날씨 : ['Rainny_state', 'Sunny_state', 'Rainny_state']
우도 함수의 값   : 0.0230


In [15]:
viterbi_path[1:][0]

(0, {
     "class" : "State",
     "distribution" : {
         "class" : "Distribution",
         "dtype" : "str",
         "name" : "DiscreteDistribution",
         "parameters" : [
             {
                 "Game" : 0.8,
                 "Fitness" : 0.2
             }
         ],
         "frozen" : false
     },
     "name" : "Rainny_state",
     "weight" : 1.0
 })